In [1]:
# 使用tf.keras搭建回归模型，数据集使用加利福尼亚的房价预测
#导入必要的库即版本

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
#import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd,sklearn,tf,keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)
matplotlib 2.2.3
numpy 1.16.4
pandas 0.23.4
sklearn 0.19.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
# 导入数据
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [3]:
# 打印数据
import pprint

pprint.pprint(housing.data[:5])
pprint.pprint(housing.target[:5])

array([[ 8.32520000e+00,  4.10000000e+01,  6.98412698e+00,
         1.02380952e+00,  3.22000000e+02,  2.55555556e+00,
         3.78800000e+01, -1.22230000e+02],
       [ 8.30140000e+00,  2.10000000e+01,  6.23813708e+00,
         9.71880492e-01,  2.40100000e+03,  2.10984183e+00,
         3.78600000e+01, -1.22220000e+02],
       [ 7.25740000e+00,  5.20000000e+01,  8.28813559e+00,
         1.07344633e+00,  4.96000000e+02,  2.80225989e+00,
         3.78500000e+01, -1.22240000e+02],
       [ 5.64310000e+00,  5.20000000e+01,  5.81735160e+00,
         1.07305936e+00,  5.58000000e+02,  2.54794521e+00,
         3.78500000e+01, -1.22250000e+02],
       [ 3.84620000e+00,  5.20000000e+01,  6.28185328e+00,
         1.08108108e+00,  5.65000000e+02,  2.18146718e+00,
         3.78500000e+01, -1.22250000e+02]])
array([4.526, 3.585, 3.521, 3.413, 3.422])


In [4]:
# 拆分训练集、测试集、验证集
from sklearn.model_selection import train_test_split

# 默认按3：1的比例拆分
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=11)

print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


### 数据归一化

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# 训练机上获得均值和方差，测试集和验证集上用相同的均值和方差
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [21]:
# 生成csv文件
output_dir = 'generate_csv'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
#切分数据，存入单独文件 
def save_to_csv(output_dir, data, name_prefix, 
                header=None, n_parts=10):
    # 生成文件名
    path_format = os.path.join(output_dir, '{}_{:02d}.csv')# 两位整数
    filenames = []
    # enumerate列出数据和数据下标
    for file_idx, row_indices in enumerate(
        np.array_split(np.arange(len(data)), n_parts)):
        # path.format生成文件名,添加至filenames中，然后打开
        part_csv = path_format.format(name_prefix, file_idx)
        filenames.append(part_csv)
        with open(part_csv, 'wt', encoding='utf-8') as f:
            if header is not None:
                f.write(header + '\n')
            for row_indice in row_indices:
                f.write(','.join([repr(col) for col in data[row_indice]]))
                f.write('\n')          
    return filenames

# 对数据进行合并
train_data = np.c_[x_train_scaled, y_train]
valid_data = np.c_[x_valid_scaled, y_valid]
test_data = np.c_[x_test_scaled, y_test]
header_cols = housing.feature_names + ['MidianHouseValue']
header_str = ','.join(header_cols)

train_filenames = save_to_csv(output_dir, train_data, 'train', 
                              header=header_str,n_parts=20)
valid_filenames = save_to_csv(output_dir, valid_data, 'valid', 
                              header=header_str,n_parts=10)
test_filenames = save_to_csv(output_dir, test_data, 'test', 
                              header=header_str,n_parts=10)

In [22]:
print(housing.feature_names)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


tf.io.decode_csv使用

In [23]:
import pprint
print('train filenames:')
pprint.pprint(train_filenames)
print('valid filenames:')
pprint.pprint(valid_filenames)
print('test filenames:')
pprint.pprint(test_filenames)

train filenames:
['generate_csv\\train_00.csv',
 'generate_csv\\train_01.csv',
 'generate_csv\\train_02.csv',
 'generate_csv\\train_03.csv',
 'generate_csv\\train_04.csv',
 'generate_csv\\train_05.csv',
 'generate_csv\\train_06.csv',
 'generate_csv\\train_07.csv',
 'generate_csv\\train_08.csv',
 'generate_csv\\train_09.csv',
 'generate_csv\\train_10.csv',
 'generate_csv\\train_11.csv',
 'generate_csv\\train_12.csv',
 'generate_csv\\train_13.csv',
 'generate_csv\\train_14.csv',
 'generate_csv\\train_15.csv',
 'generate_csv\\train_16.csv',
 'generate_csv\\train_17.csv',
 'generate_csv\\train_18.csv',
 'generate_csv\\train_19.csv']
valid filenames:
['generate_csv\\valid_00.csv',
 'generate_csv\\valid_01.csv',
 'generate_csv\\valid_02.csv',
 'generate_csv\\valid_03.csv',
 'generate_csv\\valid_04.csv',
 'generate_csv\\valid_05.csv',
 'generate_csv\\valid_06.csv',
 'generate_csv\\valid_07.csv',
 'generate_csv\\valid_08.csv',
 'generate_csv\\valid_09.csv']
test filenames:
['generate_csv\\test

In [24]:
# 读取一系列csv文件，形成一个dataset
# 1、把filenames生成dataset
# 2、读取filename生成多个datasets，merge（合并）数据
# 3、解析csv

# 生成一个文件名数据集，list_files专门处理文件名
filename_dataset = tf.data.Dataset.list_files(train_filenames)

for filename in filename_dataset:
    print(filename)

tf.Tensor(b'generate_csv\\train_14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_16.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_18.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_04.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_11.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\

In [26]:
# 读取文件,interleave函数，遍历filename_dataset中的每一个元素，对每个元素操作，操作的结果合并起来
# 对每个filename进行读取，最后合并成一个大的dataset

n_readers = 5
dataset = filename_dataset.interleave(
    # 对于每个filename构建一个数据集，按行读取文本文件API——TextLineDataset
    # skip方法控制读取文件时省略多少行，省略表头
    lambda filename: tf.data.TextLineDataset(filename).skip(1),
    # 读取文件并行度
    cycle_length = n_readers
)

# take函数只读取前15个
for line in dataset.take(15):
    print(line.numpy())
    


b'-0.09719300311107498,-1.249743071766074,0.36232962250170797,0.026906080250728295,1.033811814747154,0.045881586971778555,1.3418334617377423,-1.6353869745909178,1.832'
b'0.6363646332204844,-1.0895425985107923,0.09260902815633619,-0.20538124656801682,1.2025670451003232,-0.03630122549633783,-0.6784101660505877,0.182235342347858,2.429'
b'-0.8219588176953616,1.874166156711919,0.18212349433218608,-0.03170019246279883,-0.6011178900722581,-0.14337494105109344,1.0852205298015787,-0.8613994495208361,1.054'
b'0.04326300977263167,-1.0895425985107923,-0.38878716774583305,-0.10789864528874438,-0.6818663605100649,-0.0723871014747467,-0.8883662012710817,0.8213992340186296,1.426'
b'0.42408210084996534,0.9129633171802288,-0.04437481876046234,-0.15297213746739335,-0.24727627804141977,-0.10539166599677323,0.8612674255663844,-1.3357789003702432,3.955'
b'1.0534699704183814,-0.1283397589791022,0.13509497508586193,-0.2852867771449356,-0.37066719915986596,-0.017744041396267323,0.7586222527919203,-1.1510205879

In [27]:
# 最后得到一个大的dataset，dataset中的数据都是一个字符串，一个字符串由9个fields组成，9个fields都是数组
# 解析csv使用tf.io.decode_csv(str,record_defaults)，record_defaults参数定义字符串的类型
sample_str = '1,2,3,4,5'# 定义字符串
record_defaults = [
    tf.constant(0, dtype=tf.int32),
    0,
    np.nan,
    'hello',
    tf.constant([])# 默认float32类型
] # 定义字符串类型和默认值，当有缺失时，设成默认值,也可以使用数值定义类型

# 将一个字符串变成Tensor列表
parsed_fields = tf.io.decode_csv(sample_str, record_defaults)
print(parsed_fields)

[<tf.Tensor: id=428, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=429, shape=(), dtype=int32, numpy=2>, <tf.Tensor: id=430, shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: id=431, shape=(), dtype=string, numpy=b'4'>, <tf.Tensor: id=432, shape=(), dtype=float32, numpy=5.0>]


In [28]:
# 空值，抛出异常
try:
    parsed_fields = tf.io.decode_csv(',,,,', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

Field 4 is required but missing in record 0! [Op:DecodeCSV]


In [29]:
# 超出record_defaults的字符串，由更多的fields
try:
    parsed_fields = tf.io.decode_csv('1,2,3,4,5，6，7', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)


Field 4 in record 0 is not a valid float: 5，6，7 [Op:DecodeCSV]


In [32]:
# 解析dataset中的一行，line参数为具体的字符串，n_fields需要解析为多少个field
def parse_csv_line(line, n_fields=9):
    defs = [tf.constant(np.nan)] * n_fields
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    # 每一行中有9个元素，前8个为x，最后一个位y
    # 前八个变为一个向量
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x,y


parse_csv_line(b'-0.49303811681102094,-1.5701440182766375,-0.6933897788607161,0.16277645579446545,0.3279431630548662,-0.08806528786307917,-0.86503775291325,0.6366409215825501,2.033',n_fields=9)
    

(<tf.Tensor: id=527, shape=(8,), dtype=float32, numpy=
 array([-0.49303812, -1.570144  , -0.6933898 ,  0.16277646,  0.32794318,
        -0.08806529, -0.86503774,  0.6366409 ], dtype=float32)>,
 <tf.Tensor: id=528, shape=(1,), dtype=float32, numpy=array([2.033], dtype=float32)>)

In [33]:
# 完整读取csv文件流程
def csv_reader_dataset(filenames, n_readers=5,
                      batch_size=32, n_parse_threads=5,
                      shuffle_buffer_size=10000):
    # n_parse_threads：解析并行度
    # shuffle_buffer_size：
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filenames: tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers)
    # shuffle对数据进行混排,shuffle_buffer_size所需内存
    dataset.shuffle(shuffle_buffer_size)
    # 解析,map与interleave很类似
    dataset = dataset.map(parse_csv_line,
                         num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

train_set = csv_reader_dataset(train_filenames, batch_size=3)

for x_batch, y_batch in train_set.take(2):
    print('x:')
    pprint.pprint(x_batch)
    print('y:')
    pprint.pprint(y_batch)
    

x:
<tf.Tensor: id=614, shape=(3, 8), dtype=float32, numpy=
array([[ 0.63034356,  1.8741661 , -0.06713215, -0.12543367, -0.19737554,
        -0.02272263, -0.69240725,  0.72652334],
       [ 0.63034356,  1.8741661 , -0.06713215, -0.12543367, -0.19737554,
        -0.02272263, -0.69240725,  0.72652334],
       [ 0.63034356,  1.8741661 , -0.06713215, -0.12543367, -0.19737554,
        -0.02272263, -0.69240725,  0.72652334]], dtype=float32)>
y:
<tf.Tensor: id=615, shape=(3, 1), dtype=float32, numpy=
array([[2.419],
       [2.419],
       [2.419]], dtype=float32)>
x:
<tf.Tensor: id=618, shape=(3, 8), dtype=float32, numpy=
array([[ 0.63034356,  1.8741661 , -0.06713215, -0.12543367, -0.19737554,
        -0.02272263, -0.69240725,  0.72652334],
       [ 0.63034356,  1.8741661 , -0.06713215, -0.12543367, -0.19737554,
        -0.02272263, -0.69240725,  0.72652334],
       [ 1.6312258 ,  0.35226166,  0.04080576, -0.14088951, -0.4632104 ,
        -0.06751624, -0.82771224,  0.59669316]], dtype=float32)

In [35]:
batch_size = 32
train_set = csv_reader_dataset(train_filenames, batch_size=batch_size)
valid_set = csv_reader_dataset(valid_filenames, batch_size=batch_size)
test_set = csv_reader_dataset(test_filenames, batch_size=batch_size)

与keras结合

In [36]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=[8]),
    #最后一层一个神经元
    keras.layers.Dense(1),
])
# 编译，目标函数使用均方差，优化方法使用随机梯度下降，也可以自己定义
model.compile(loss='mean_squared_error', optimizer='sgd')
# 使用earlystopping
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)]


history = model.fit(train_set,
                   validation_data = valid_set,
                    # dataset不停的产生数据，fit函数需要知道跑多少次才是一个周期（epoch）
                    # 即遍历一次需要多少次迭代
                    steps_per_epoch = 11160 // batch_size,
                    validation_steps = 3870 // batch_size,
                   epochs=100, callbacks=callbacks)

Epoch 1/100
348/348 [==============================] - 1s 3ms/step - loss: 1.7823 - val_loss: 0.6837
Epoch 2/100
348/348 [==============================] - 1s 2ms/step - loss: 0.5520 - val_loss: 0.4828
Epoch 3/100
348/348 [==============================] - 0s 1ms/step - loss: 0.4812 - val_loss: 0.4684
Epoch 4/100
348/348 [==============================] - 0s 1ms/step - loss: 0.4498 - val_loss: 0.5043
Epoch 5/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4311 - val_loss: 0.4015
Epoch 6/100
348/348 [==============================] - 0s 1ms/step - loss: 0.4156 - val_loss: 0.3939
Epoch 7/100
348/348 [==============================] - 0s 1ms/step - loss: 0.4154 - val_loss: 0.4388
Epoch 8/100
348/348 [==============================] - 0s 1ms/step - loss: 0.3965 - val_loss: 0.3783
Epoch 9/100
348/348 [==============================] - 0s 1ms/step - loss: 0.3890 - val_loss: 0.3957
Epoch 10/100
348/348 [==============================] - 0s 1ms/step - loss: 0.3889 - val_lo

In [37]:
model.evaluate(test_set,steps= 5160 // batch_size)

161/161 [==============================] - 0s 1ms/step - loss: 0.2794


0.27937541456071674